In [42]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load MNIST data from sklearn's openml
mnist = fetch_openml('mnist_784', version=1)
x, y = mnist['data'], mnist['target']

# Normalize pixel values to the range [0, 1]
x = x / 255.0

# Split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Convert labels to integers and one-hot encode them
y_train = y_train.astype(int).to_numpy()  # Convert to NumPy array
y_test = y_test.astype(int).to_numpy()

one_hot_encoder = OneHotEncoder(sparse_output=False)
y_train = one_hot_encoder.fit_transform(y_train.reshape(-1, 1))
y_test = one_hot_encoder.transform(y_test.reshape(-1, 1))

# Neural network parameters
input_size = 784
h1_size = 512
h2_size = 256
h3_size = 128
h4_size = 64
output_size = 10
learning_rate = 0.01
epochs = 10

# Initialize weights and biases with values and gradients (dual numbers)
W1 = np.random.randn(input_size, h1_size) * np.sqrt(2. / input_size)
b1 = np.zeros((1, h1_size))
W2 = np.random.randn(h1_size, h2_size) * np.sqrt(2. / h1_size)
b2 = np.zeros((1, h2_size))
W3 = np.random.randn(h2_size, h3_size) * np.sqrt(2. / h2_size)
b3 = np.zeros((1, h3_size))
W4 = np.random.randn(h3_size, h4_size) * np.sqrt(2. / h3_size)
b4 = np.zeros((1, h4_size))
W5 = np.random.randn(h4_size, output_size) * np.sqrt(2. / h4_size)
b5 = np.zeros((1, output_size))

# Activation functions
def sigmoid(x):
    x = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Loss function
def cross_entropy_loss(predictions, targets):
    return -np.mean(np.sum(targets * np.log(predictions + 1e-9), axis=1))

# Forward pass
def forward(X):
    z1 = np.dot(X, W1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = sigmoid(z2)
    z3 = np.dot(a2, W3) + b3
    a3 = sigmoid(z3)
    z4 = np.dot(a3, W4) + b4
    a4 = sigmoid(z4)
    z5 = np.dot(a4, W5) + b5
    a5 = softmax(z5)
    return z1, a1, z2, a2, z3, a3, z4, a4, z5, a5

# Backward pass
def backward(X, y, z1, a1, z2, a2, z3, a3, z4, a4, z5, a5):
    global W1, b1, W2, b2, W3, b3, W4, b4, W5, b5

    # Output layer error
    error_output = a5 - y
    dW5 = np.dot(a4.T, error_output)
    db5 = np.sum(error_output, axis=0, keepdims=True)

    # Hidden layer error
    error_hidden = np.dot(error_output, W4.T) * sigmoid_derivative(a4)
    dW4 = np.dot(a3.T, error_hidden)
    db4 = np.sum(error_hidden, axis=0, keepdims=True)

    error_hidden = np.dot(error_output, W3.T) * sigmoid_derivative(a3)
    dW3 = np.dot(a2.T, error_hidden)
    db3 = np.sum(error_hidden, axis=0, keepdims=True)   

    error_hidden = np.dot(error_output, W2.T) * sigmoid_derivative(a2)
    dW2 = np.dot(a1.T, error_hidden)
    db2 = np.sum(error_hidden, axis=0, keepdims=True)

    error_hidden = np.dot(error_output, W1.T) * sigmoid_derivative(a1)
    dW1 = np.dot(X.T, error_hidden)
    db1 = np.sum(error_hidden, axis=0, keepdims=True)

    # Update weights and biases
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W3 -= learning_rate * dW3
    b3 -= learning_rate * db3
    W4 -= learning_rate * dW4
    b4 -= learning_rate * db4
    W5 -= learning_rate * dW5
    b5 -= learning_rate * db5

# Training loop
for epoch in range(epochs):
    z1, a1, z2, a2, z3, a3, z4, a4, z5, a5 = forward(x_train)
    loss = cross_entropy_loss(a5, y_train)
    backward(x_train, y_train, z1, a1, z2, a2, z3, a3, z4, a4, z5, a5)
    
    if epoch % 1 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")

# Testing accuracy
_, _, _, _, _, _, _, _, _, a5_test = forward(x_test)
predictions = np.argmax(a5_test, axis=1)
accuracy = np.mean(predictions == np.argmax(y_test, axis=1))
print(f"Test Accuracy: {accuracy * 100:.2f}%")

ValueError: shapes (56000,10) and (64,128) not aligned: 10 (dim 1) != 64 (dim 0)

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Load and preprocess the MNIST data
mnist = fetch_openml('mnist_784', version=1)
x, y = mnist['data'], mnist['target']

# Normalize pixel values to the range [0, 1]
x = x / 255.0

# Split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Convert labels to integers and one-hot encode them
y_train = y_train.astype(int).to_numpy()
y_test = y_test.astype(int).to_numpy()

one_hot_encoder = OneHotEncoder(sparse_output=False)
y_train = one_hot_encoder.fit_transform(y_train.reshape(-1, 1))
y_test = one_hot_encoder.transform(y_test.reshape(-1, 1))

# Convert data to NumPy arrays before creating PyTorch tensors
x_train_tensor = torch.tensor(x_train.to_numpy(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test.to_numpy(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# Initialize the model, loss function, and optimizer
input_size = 784
hidden_size = 64
output_size = 10
learning_rate = 0.01
epochs = 10

model = NeuralNetwork(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    # Forward pass
    outputs = model(x_train_tensor)
    loss = criterion(outputs, torch.argmax(y_train_tensor, dim=1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 1 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Testing accuracy
with torch.no_grad():
    outputs_test = model(x_test_tensor)
    _, predicted = torch.max(outputs_test, 1)
    accuracy = (predicted == torch.argmax(y_test_tensor, dim=1)).float().mean()
    print(f"Test Accuracy: {accuracy.item() * 100:.2f}%")

Epoch 1/10, Loss: 2.3037
Epoch 2/10, Loss: 2.3037
Epoch 3/10, Loss: 2.3037
Epoch 4/10, Loss: 2.3037
Epoch 5/10, Loss: 2.3036
Epoch 6/10, Loss: 2.3036
Epoch 7/10, Loss: 2.3036
Epoch 8/10, Loss: 2.3036
Epoch 9/10, Loss: 2.3036
Epoch 10/10, Loss: 2.3036
Test Accuracy: 9.13%


In [17]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load MNIST data from sklearn's openml
mnist = fetch_openml('mnist_784', version=1)
x, y = mnist['data'], mnist['target']

# Normalize pixel values to the range [0, 1]
x = x / 255.0

# Split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Convert labels to integers and one-hot encode them
y_train = y_train.astype(int).to_numpy()  # Convert to NumPy array
y_test = y_test.astype(int).to_numpy()

one_hot_encoder = OneHotEncoder(sparse_output=False)
y_train = one_hot_encoder.fit_transform(y_train.reshape(-1, 1))
y_test = one_hot_encoder.transform(y_test.reshape(-1, 1))

# Neural network parameters
input_size = 784
hidden_size = 64
output_size = 10
learning_rate = 0.01
epochs = 10
epsilon = 1e-4  # Small value for finite differences

# Initialize weights and biases
W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2. / input_size)
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2. / hidden_size)
b2 = np.zeros((1, output_size))

# Activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Loss function
def cross_entropy_loss(predictions, targets):
    return -np.mean(np.sum(targets * np.log(predictions + 1e-9), axis=1))

# Forward pass
def forward(X):
    z1 = np.dot(X, W1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = softmax(z2)
    return z1, a1, z2, a2

# Forward Gradient Descent
def forward_gradient_descent(X, y, z1, a1, z2, a2):
    global W1, b1, W2, b2

    # Gradients for W2 and b2 using finite differences
    dW2 = np.zeros_like(W2)
    db2 = np.zeros_like(b2)

    for i in range(W2.shape[0]):
        for j in range(W2.shape[1]):
            W2[i, j] += epsilon
            _, _, _, a2_perturbed = forward(X)
            loss_perturbed = cross_entropy_loss(a2_perturbed, y)
            dW2[i, j] = (loss_perturbed - cross_entropy_loss(a2, y)) / epsilon
            W2[i, j] -= epsilon

    for j in range(b2.shape[1]):
        b2[0, j] += epsilon
        _, _, _, a2_perturbed = forward(X)
        loss_perturbed = cross_entropy_loss(a2_perturbed, y)
        db2[0, j] = (loss_perturbed - cross_entropy_loss(a2, y)) / epsilon
        b2[0, j] -= epsilon

    # Gradients for W1 and b1
    dW1 = np.zeros_like(W1)
    db1 = np.zeros_like(b1)

    for i in range(W1.shape[0]):
        for j in range(W1.shape[1]):
            W1[i, j] += epsilon
            _, a1_perturbed, _, a2_perturbed = forward(X)
            loss_perturbed = cross_entropy_loss(a2_perturbed, y)
            dW1[i, j] = (loss_perturbed - cross_entropy_loss(a2, y)) / epsilon
            W1[i, j] -= epsilon

    for j in range(b1.shape[1]):
        b1[0, j] += epsilon
        _, a1_perturbed, _, a2_perturbed = forward(X)
        loss_perturbed = cross_entropy_loss(a2_perturbed, y)
        db1[0, j] = (loss_perturbed - cross_entropy_loss(a2, y)) / epsilon
        b1[0, j] -= epsilon

    # Update weights and biases
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

# Training loop with forward gradient descent
for epoch in range(epochs):
    z1, a1, z2, a2 = forward(x_train)
    loss = cross_entropy_loss(a2, y_train)
    forward_gradient_descent(x_train, y_train, z1, a1, z2, a2)
    
    if epoch % 1 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")

# Testing accuracy
_, _, _, a2_test = forward(x_train)
predictions = np.argmax(a2_test, axis=1)
accuracy = np.mean(predictions == np.argmax(y_train, axis=1))
print(f"Training Accuracy on Synthetic Data: {accuracy * 100:.2f}%")

Epoch 1/10, Loss: 2.5652


KeyboardInterrupt: 

In [29]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load MNIST data from sklearn's openml
mnist = fetch_openml('mnist_784', version=1)
x, y = mnist['data'], mnist['target']

# Normalize pixel values to the range [0, 1]
x = x / 255.0

# Split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Convert labels to integers and one-hot encode them
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
y_train = y_train.astype(int).to_numpy()  # Convert to NumPy array
y_test = y_test.astype(int).to_numpy()

one_hot_encoder = OneHotEncoder(sparse_output=False)
y_train = one_hot_encoder.fit_transform(y_train.reshape(-1, 1))
y_test = one_hot_encoder.transform(y_test.reshape(-1, 1))

# Neural network parameters
input_size = 784
hidden_size = 64
output_size = 10
learning_rate = 0.01

# Initialize weights and biases
W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2. / input_size)
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2. / hidden_size)
b2 = np.zeros((1, output_size))

# Activation functions and their derivatives
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Loss function
def cross_entropy_loss(predictions, targets):
    return -np.mean(np.sum(targets * np.log(predictions + 1e-9), axis=1))

# Forward-mode AD function to calculate gradients directly in the forward pass
def forward_ad(X):
    global W1, b1, W2, b2

    # Forward pass for values
    z1 = np.dot(X, W1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = softmax(z2)
    
    # Forward-mode AD for gradients
    # Initialize gradients with respect to each parameter
    dW1 = np.zeros_like(W1)
    db1 = np.zeros_like(b1)
    dW2 = np.zeros_like(W2)
    db2 = np.zeros_like(b2)
    
    # Compute gradients for W2 and b2
    for i in range(W2.shape[0]):
        for j in range(W2.shape[1]):
            # Perturb W2[i, j] and compute the gradient directly
            dW2[i, j] = np.sum(a1[:, i] * (a2[:, j] - y_train[:, j])) / X.shape[0]
    
    db2[0, :] = np.sum(a2 - y_train, axis=0) / X.shape[0]
    
    # Compute gradients for W1 and b1
    delta_hidden = (np.dot(a2 - y_train, W2.T) * sigmoid_derivative(a1))  # Backpropagated error to hidden layer
    
    for i in range(W1.shape[0]):
        for j in range(W1.shape[1]):
            # Perturb W1[i, j] and compute the gradient directly
            dW1[i, j] = np.sum(X[:, i] * delta_hidden[:, j]) / X.shape[0]
    
    db1[0, :] = np.sum(delta_hidden, axis=0) / X.shape[0]
    
    return a2, dW1, db1, dW2, db2

# Training loop with forward-mode AD
epochs = 10
for epoch in range(epochs):
    # Perform forward pass and get gradients
    a2, dW1, db1, dW2, db2 = forward_ad(x_train)
    loss = cross_entropy_loss(a2, y_train)
    
    # Gradient descent update
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    
    # Print loss at each epoch
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")

# Test the training accuracy on synthetic data
a2, _, _, _, _ = forward_ad(x_train)
predictions = np.argmax(a2, axis=1)
accuracy = np.mean(predictions == np.argmax(y_train, axis=1))
print(f"Training Accuracy on Synthetic Data: {accuracy * 100:.2f}%")

Epoch 1/10, Loss: 2.6984
Epoch 2/10, Loss: 2.6815
Epoch 3/10, Loss: 2.6654
Epoch 4/10, Loss: 2.6501
Epoch 5/10, Loss: 2.6354
Epoch 6/10, Loss: 2.6214
Epoch 7/10, Loss: 2.6080
Epoch 8/10, Loss: 2.5952
Epoch 9/10, Loss: 2.5829
Epoch 10/10, Loss: 2.5711
Training Accuracy on Synthetic Data: 9.19%


In [39]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load MNIST data from sklearn's openml
mnist = fetch_openml('mnist_784', version=1)
x, y = mnist['data'], mnist['target']

# Normalize pixel values to the range [0, 1]
x = x / 255.0

# Split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Convert labels to integers and one-hot encode them
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
y_train = y_train.astype(int).to_numpy()  # Convert to NumPy array
y_test = y_test.astype(int).to_numpy()

one_hot_encoder = OneHotEncoder(sparse_output=False)
y_train = one_hot_encoder.fit_transform(y_train.reshape(-1, 1))
y_test = one_hot_encoder.transform(y_test.reshape(-1, 1))

# Neural network parameters
input_size = 784
h1_size = 512
h2_size = 256
h3_size = 128
h4_size = 64
output_size = 10
learning_rate = 0.01

# Initialize weights and biases with values and gradients (dual numbers)
W1 = np.random.randn(input_size, h1_size) * np.sqrt(2. / input_size)
b1 = np.zeros((1, h1_size))
W2 = np.random.randn(h1_size, h2_size) * np.sqrt(2. / h1_size)
b2 = np.zeros((1, h2_size))
W3 = np.random.randn(h2_size, h3_size) * np.sqrt(2. / h2_size)
b3 = np.zeros((1, h3_size))
W4 = np.random.randn(h3_size, h4_size) * np.sqrt(2. / h3_size)
b4 = np.zeros((1, h4_size))
W5 = np.random.randn(h4_size, output_size) * np.sqrt(2. / h4_size)
b5 = np.zeros((1, output_size))

# Activation functions and their derivatives
def sigmoid(x):
    x = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Loss function
def cross_entropy_loss(predictions, targets):
    return -np.mean(np.sum(targets * np.log(predictions + 1e-9), axis=1))

# Forward-mode AD function
def forward_ad(X):
    global W1, b1, W2, b2, W3, b3, W4, b4, W5, b5
    # Forward pass for values and derivatives
    
    # Layer 1: Input to Hidden Layer
    z1 = np.dot(X, W1) + b1  # regular forward pass
    a1 = sigmoid(z1)         # activation

    # Gradient of z1 w.r.t. W1, b1
    dW1 = np.dot(X.T, sigmoid_derivative(a1))
    db1 = np.sum(sigmoid_derivative(a1), axis=0, keepdims=True)
    
    # Layer 2: Hidden to Output Layer
    z2 = np.dot(a1, W2) + b2
    a2 = sigmoid(z2)

    dW2 = np.dot(a1.T, sigmoid_derivative(a2))
    db2 = np.sum(sigmoid_derivative(a2), axis=0, keepdims=True)

    # Layer 3: Hidden to Output Layer
    z3 = np.dot(a2, W3) + b3
    a3 = sigmoid(z3)

    dW3 = np.dot(a2.T, sigmoid_derivative(a3))
    db3 = np.sum(sigmoid_derivative(a3), axis=0, keepdims=True)

    # Layer 4: Hidden to Output Layer
    z4 = np.dot(a3, W4) + b4
    a4 = sigmoid(z4)

    dW4 = np.dot(a3.T, sigmoid_derivative(a4))
    db4 = np.sum(sigmoid_derivative(a4), axis=0, keepdims=True)

    # Layer 5: Hidden to Output Layer
    z5 = np.dot(a4, W5) + b5
    a5 = softmax(z5)

    dW5 = np.dot(a4.T, (a5 - y_train))
    db5 = np.sum(a5 - y_train, axis=0, keepdims=True)

    return a5, dW1, db1, dW2, db2, dW3, db3, dW4, db4, dW5, db5

# Training loop with forward-mode AD
epochs = 10
for epoch in range(epochs):
    # Perform forward pass and get gradients
    a5, dW1, db1, dW2, db2, dW3, db3, dW4, db4, dW5, db5 = forward_ad(x_train)
    loss = cross_entropy_loss(a5, y_train)
    
    # Gradient descent update
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W3 -= learning_rate * dW3
    b3 -= learning_rate * db3
    W4 -= learning_rate * dW4
    b4 -= learning_rate * db4
    W5 -= learning_rate * dW5
    b5 -= learning_rate * db5
    
    # Print loss at each epoch
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")

# Test the training accuracy on synthetic data
a5, _, _, _, _, _, _, _, _, _, _ = forward_ad(x_train)
predictions = np.argmax(a5, axis=1)
accuracy = np.mean(predictions == np.argmax(y_train, axis=1))
print(f"Training Accuracy on Synthetic Data: {accuracy * 100:.2f}%")

Epoch 1/10, Loss: 2.4263
Epoch 2/10, Loss: 15.8614
Epoch 3/10, Loss: 15.6920
Epoch 4/10, Loss: 15.1998
Epoch 5/10, Loss: 16.9301
Epoch 6/10, Loss: 16.8340
Epoch 7/10, Loss: 18.3289
Epoch 8/10, Loss: 17.7703
Epoch 9/10, Loss: 18.8570
Epoch 10/10, Loss: 18.6998
Training Accuracy on Synthetic Data: 9.79%


In [46]:
import torch
import torch.nn as nn
import time
from functorch import jvp, make_functional

# Function to create a feedforward neural network
def create_network(input_size, layer_sizes, output_size):
    layers = []
    for i, layer_size in enumerate(layer_sizes):
        if i == 0:
            layers.append(nn.Linear(input_size, layer_size))
        else:
            layers.append(nn.Linear(layer_sizes[i - 1], layer_size))
        layers.append(nn.ReLU())
    layers.append(nn.Linear(layer_sizes[-1], output_size))
    return nn.Sequential(*layers)

# Function to benchmark backpropagation
def benchmark_backprop(model, input_data, target_data, criterion):
    # Forward pass
    output = model(input_data)
    loss = criterion(output, target_data)
    
    # Backward pass
    start_time = time.time()
    loss.backward()
    end_time = time.time()
    
    return end_time - start_time

# Function to benchmark forward-mode AD using jvp
def benchmark_forward_ad(model, input_data, target_data, criterion):
    # Define function for forward pass with unpacking
    def forward_fn(*params):
        # Assign parameters to model
        with torch.no_grad():
            for p, p_val in zip(model.parameters(), params):
                p.data.copy_(p_val)
        output = model(input_data)
        loss = criterion(output, target_data)
        return loss

    # Create dummy vector for jvp and convert params to tuple
    params = tuple(model.parameters())
    dummy_vec = tuple(torch.ones_like(p) for p in params)

    # Forward pass and JVP (forward-mode AD)
    start_time = time.time()
    _, jvp_val = jvp(forward_fn, params, dummy_vec)
    end_time = time.time()
    
    return end_time - start_time

# Function to run benchmarks for different network sizes
def run_benchmarks(input_size, output_size, batch_size, layer_sizes_list, num_trials=5):
    input_data = torch.randn(batch_size, input_size)
    target_data = torch.randint(0, output_size, (batch_size,))
    criterion = nn.CrossEntropyLoss()

    results = []
    for layer_sizes in layer_sizes_list:
        print(f"Benchmarking network with layer sizes: {layer_sizes}")
        
        # Initialize model
        model = create_network(input_size, layer_sizes, output_size)
        
        # Run benchmarks
        backprop_times = []
        forward_ad_times = []
        for _ in range(num_trials):
            # Backpropagation
            backprop_time = benchmark_backprop(model, input_data, target_data, criterion)
            backprop_times.append(backprop_time)
            
            # Forward-mode AD
            forward_ad_time = benchmark_forward_ad(model, input_data, target_data, criterion)
            forward_ad_times.append(forward_ad_time)
        
        # Store average times
        avg_backprop_time = sum(backprop_times) / num_trials
        avg_forward_ad_time = sum(forward_ad_times) / num_trials
        results.append({
            "Layer Sizes": layer_sizes,
            "Backprop Time (s)": avg_backprop_time,
            "Forward AD Time (s)": avg_forward_ad_time
        })
        
    return results

# Define network configurations to test
input_size = 784
output_size = 10
batch_size = 64
layer_sizes_list = [
    [128],         # Single hidden layer
    [256, 128],    # Two hidden layers
    [512, 256, 128],  # Three hidden layers
    [512, 256, 128, 64]  # Four hidden layers
]

# Run benchmarks
results = run_benchmarks(input_size, output_size, batch_size, layer_sizes_list)

# Display results
import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)

Benchmarking network with layer sizes: [128]


/var/folders/2m/zt75k_js249d_qcmywz2vb0r0000gp/T/ipykernel_25034/1375182697.py:49: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.jvp` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.func.jvp` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  _, jvp_val = jvp(forward_fn, params, dummy_vec)


RuntimeError: During a grad (vjp, jvp, grad, etc) transform, the function provided attempted to call in-place operation (aten::copy_) that would mutate a captured Tensor. This is not supported; please rewrite the function being transformed to explicitly accept the mutated Tensor(s) as inputs.

In [49]:
import torch
import torch.nn as nn
import time
from functorch import jvp, make_functional

# Function to create a feedforward neural network
def create_network(input_size, layer_sizes, output_size):
    layers = []
    for i, layer_size in enumerate(layer_sizes):
        if i == 0:
            layers.append(nn.Linear(input_size, layer_size))
        else:
            layers.append(nn.Linear(layer_sizes[i - 1], layer_size))
        layers.append(nn.ReLU())
    layers.append(nn.Linear(layer_sizes[-1], output_size))
    return nn.Sequential(*layers)

# Function to benchmark backpropagation
def benchmark_backprop(model, input_data, target_data, criterion):
    # Forward pass
    output = model(input_data)
    loss = criterion(output, target_data)
    
    # Backward pass
    start_time = time.time()
    loss.backward()
    end_time = time.time()
    
    return end_time - start_time

# Function to benchmark forward-mode AD using jvp
def benchmark_forward_ad(model, input_data, target_data, criterion):
    # Convert model to functional form
    fmodel, params = make_functional(model)

    # Define function for forward pass in functional form
    def forward_fn(params):
        output = fmodel(params, input_data)
        loss = criterion(output, target_data)
        return loss

    # Create dummy vector for jvp
    dummy_vec = tuple(torch.ones_like(p) for p in params)

    # Forward pass and JVP (forward-mode AD)
    start_time = time.time()
    _, jvp_val = jvp(forward_fn, (params,), (dummy_vec,))
    end_time = time.time()
    
    return end_time - start_time

# Function to run benchmarks for different network sizes
def run_benchmarks(input_size, output_size, batch_size, layer_sizes_list, num_trials=5):
    input_data = torch.randn(batch_size, input_size)
    target_data = torch.randint(0, output_size, (batch_size,))
    criterion = nn.CrossEntropyLoss()

    results = []
    for layer_sizes in layer_sizes_list:
        print(f"Benchmarking network with layer sizes: {layer_sizes}")
        
        # Initialize model
        model = create_network(input_size, layer_sizes, output_size)
        
        # Run benchmarks
        backprop_times = []
        forward_ad_times = []
        for _ in range(num_trials):
            # Backpropagation
            backprop_time = benchmark_backprop(model, input_data, target_data, criterion)
            backprop_times.append(backprop_time)
            
            # Forward-mode AD
            forward_ad_time = benchmark_forward_ad(model, input_data, target_data, criterion)
            forward_ad_times.append(forward_ad_time)
        
        # Store average times
        avg_backprop_time = sum(backprop_times) / num_trials
        avg_forward_ad_time = sum(forward_ad_times) / num_trials
        results.append({
            "Layer Sizes": layer_sizes,
            "Backprop Time (s)": avg_backprop_time,
            "Forward AD Time (s)": avg_forward_ad_time
        })
        
    return results

# Define network configurations to test
input_size = 784
output_size = 10
batch_size = 512
layer_sizes_list = [
    [128],         # Single hidden layer
    [256, 128],    # Two hidden layers
    [512, 256, 128],  # Three hidden layers
    [512, 256, 128, 64]  # Four hidden layers
]

# Run benchmarks
results = run_benchmarks(input_size, output_size, batch_size, layer_sizes_list)

# Display results
import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)

Benchmarking network with layer sizes: [128]
Benchmarking network with layer sizes: [256, 128]
Benchmarking network with layer sizes: [512, 256, 128]
Benchmarking network with layer sizes: [512, 256, 128, 64]
           Layer Sizes  Backprop Time (s)  Forward AD Time (s)
0                [128]           0.000453             0.002594
1           [256, 128]           0.000748             0.002718
2      [512, 256, 128]           0.002067             0.005176
3  [512, 256, 128, 64]           0.001675             0.003887


/var/folders/2m/zt75k_js249d_qcmywz2vb0r0000gp/T/ipykernel_25034/2525052501.py:34: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.make_functional` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.func.functional_call` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  fmodel, params = make_functional(model)
/var/folders/2m/zt75k_js249d_qcmywz2vb0r0000gp/T/ipykernel_25034/2525052501.py:47: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.jvp` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.func.jvp` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  _, jvp_v

In [50]:
import torch
import torch.nn as nn
import time
from functorch import jvp, make_functional
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Function to create a feedforward neural network
def create_network(input_size, layer_sizes, output_size):
    layers = []
    for i, layer_size in enumerate(layer_sizes):
        if i == 0:
            layers.append(nn.Linear(input_size, layer_size))
        else:
            layers.append(nn.Linear(layer_sizes[i - 1], layer_size))
        layers.append(nn.ReLU())
    layers.append(nn.Linear(layer_sizes[-1], output_size))
    return nn.Sequential(*layers)

# Function to benchmark backpropagation
def benchmark_backprop(model, input_data, target_data, criterion):
    # Forward pass
    output = model(input_data)
    loss = criterion(output, target_data)
    
    # Backward pass
    start_time = time.time()
    loss.backward()
    end_time = time.time()
    
    return end_time - start_time

# Function to benchmark forward-mode AD using jvp
def benchmark_forward_ad(model, input_data, target_data, criterion):
    # Convert model to functional form
    fmodel, params = make_functional(model)

    # Define function for forward pass in functional form
    def forward_fn(params):
        output = fmodel(params, input_data)
        loss = criterion(output, target_data)
        return loss

    # Create dummy vector for jvp
    dummy_vec = tuple(torch.ones_like(p) for p in params)

    # Forward pass and JVP (forward-mode AD)
    start_time = time.time()
    _, jvp_val = jvp(forward_fn, (params,), (dummy_vec,))
    end_time = time.time()
    
    return end_time - start_time

# Function to run benchmarks for different network sizes
def run_benchmarks(input_data, target_data, input_size, output_size, layer_sizes_list, num_trials=5):
    criterion = nn.CrossEntropyLoss()

    results = []
    for layer_sizes in layer_sizes_list:
        print(f"Benchmarking network with layer sizes: {layer_sizes}")
        
        # Initialize model
        model = create_network(input_size, layer_sizes, output_size)
        
        # Run benchmarks
        backprop_times = []
        forward_ad_times = []
        for _ in range(num_trials):
            # Backpropagation
            backprop_time = benchmark_backprop(model, input_data, target_data, criterion)
            backprop_times.append(backprop_time)
            
            # Forward-mode AD
            forward_ad_time = benchmark_forward_ad(model, input_data, target_data, criterion)
            forward_ad_times.append(forward_ad_time)
        
        # Store average times
        avg_backprop_time = sum(backprop_times) / num_trials
        avg_forward_ad_time = sum(forward_ad_times) / num_trials
        results.append({
            "Layer Sizes": layer_sizes,
            "Backprop Time (s)": avg_backprop_time,
            "Forward AD Time (s)": avg_forward_ad_time
        })
        
    return results

# Load and preprocess the MNIST dataset
def load_mnist_data():
    mnist = fetch_openml('mnist_784', version=1)
    x, y = mnist['data'], mnist['target']
    x = x / 255.0  # Normalize to [0, 1]
    y = LabelEncoder().fit_transform(y)

    # Split into train and test sets
    x_train, _, y_train, _ = train_test_split(x, y, test_size=0.9, random_state=42)
    
    # Convert to PyTorch tensors
    x_train = torch.tensor(x_train.values, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    
    return x_train, y_train

# Define network configurations to test
input_size = 784
output_size = 10
layer_sizes_list = [
    [128],         # Single hidden layer
    [256, 128],    # Two hidden layers
    [512, 256, 128],  # Three hidden layers
    [512, 256, 128, 64]  # Four hidden layers
]

# Load MNIST data
input_data, target_data = load_mnist_data()

# Run benchmarks
results = run_benchmarks(input_data, target_data, input_size, output_size, layer_sizes_list)

# Display results
import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)

Benchmarking network with layer sizes: [128]
Benchmarking network with layer sizes: [256, 128]


/var/folders/2m/zt75k_js249d_qcmywz2vb0r0000gp/T/ipykernel_25034/1272575570.py:38: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.make_functional` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.func.functional_call` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  fmodel, params = make_functional(model)
/var/folders/2m/zt75k_js249d_qcmywz2vb0r0000gp/T/ipykernel_25034/1272575570.py:51: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.jvp` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.func.jvp` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  _, jvp_v

Benchmarking network with layer sizes: [512, 256, 128]
Benchmarking network with layer sizes: [512, 256, 128, 64]
           Layer Sizes  Backprop Time (s)  Forward AD Time (s)
0                [128]           0.005485             0.014256
1           [256, 128]           0.014681             0.033693
2      [512, 256, 128]           0.023862             0.053527
3  [512, 256, 128, 64]           0.023846             0.051046


In [2]:
import torch
import torch.nn as nn
import time
from functorch import jacfwd, make_functional
from torch.utils._pytree import tree_flatten, tree_unflatten
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Function to create a feedforward neural network
def create_network(input_size, layer_sizes, output_size):
    layers = []
    for i, layer_size in enumerate(layer_sizes):
        if i == 0:
            layers.append(nn.Linear(input_size, layer_size))
        else:
            layers.append(nn.Linear(layer_sizes[i - 1], layer_size))
        layers.append(nn.ReLU())
    layers.append(nn.Linear(layer_sizes[-1], output_size))
    return nn.Sequential(*layers)

# Function to train with backpropagation
def train_backprop(model, input_data, target_data, criterion, optimizer, epochs):
    start_time = time.time()
    for _ in range(epochs):
        optimizer.zero_grad()
        output = model(input_data)
        loss = criterion(output, target_data)
        loss.backward()
        optimizer.step()
    end_time = time.time()
    return end_time - start_time

# Function to train with forward-mode AD using jvp
def train_forward_ad(model, input_data, target_data, criterion, epochs, learning_rate):
    # Convert model to functional form
    fmodel, params = make_functional(model)

    def forward_fn(params):
        output = fmodel(params, input_data)
        loss = criterion(output, target_data)
        return loss  # return a scalar loss

    # Flatten params to avoid nested structure issues
    flat_params, params_spec = tree_flatten(params)

    start_time = time.time()
    for _ in range(epochs):
        # Compute gradients for forward-mode AD
        grads = jacfwd(forward_fn)(params)

        # Flatten gradients to match flat_params
        flat_grads, _ = tree_flatten(grads)

        # Update parameters (flat updates)
        flat_params = [p - learning_rate * g for p, g in zip(flat_params, flat_grads)]

        # Reconstruct parameter structure
        params = tree_unflatten(flat_params, params_spec)

    end_time = time.time()
    return end_time - start_time

# Function to run benchmarks for different network sizes
def run_benchmarks(input_data, target_data, input_size, output_size, layer_sizes_list, epochs, num_trials=5):
    criterion = nn.CrossEntropyLoss()
    results = []

    for layer_sizes in layer_sizes_list:
        print(f"Benchmarking network with layer sizes: {layer_sizes}")
        
        # Initialize model
        model = create_network(input_size, layer_sizes, output_size)
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

        # Run benchmarks
        backprop_times = []
        forward_ad_times = []
        for _ in range(num_trials):
            # Backpropagation training
            backprop_time = train_backprop(model, input_data, target_data, criterion, optimizer, epochs)
            backprop_times.append(backprop_time)
            
            # Forward-mode AD training
            forward_ad_time = train_forward_ad(model, input_data, target_data, criterion, epochs, learning_rate)
            forward_ad_times.append(forward_ad_time)
        
        # Store average times
        avg_backprop_time = sum(backprop_times) / num_trials
        avg_forward_ad_time = sum(forward_ad_times) / num_trials
        results.append({
            "Layer Sizes": layer_sizes,
            "Backprop Total Time (s)": avg_backprop_time,
            "Forward AD Total Time (s)": avg_forward_ad_time,
            "Avg Backprop Epoch Time (s)": avg_backprop_time / epochs,
            "Avg Forward AD Epoch Time (s)": avg_forward_ad_time / epochs
        })
        
    return results

# Load and preprocess the MNIST dataset
def load_mnist_data():
    mnist = fetch_openml('mnist_784', version=1)
    x, y = mnist['data'], mnist['target']
    x = x / 255.0  # Normalize to [0, 1]
    y = LabelEncoder().fit_transform(y)

    # Split into train and test sets
    x_train, _, y_train, _ = train_test_split(x, y, test_size=0.9, random_state=42)
    
    # Convert to PyTorch tensors
    x_train = torch.tensor(x_train.values, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    
    return x_train, y_train

# Define network configurations to test
input_size = 784
output_size = 10
layer_sizes_list = [
    [128],         # Single hidden layer
    [256, 128],    # Two hidden layers
    [512, 256, 128],  # Three hidden layers
    [512, 256, 128, 64]  # Four hidden layers
]

# Set hyperparameters
learning_rate = 0.01
epochs = 5  # Adjust as needed for longer training

# Load MNIST data
input_data, target_data = load_mnist_data()

# Run benchmarks
results = run_benchmarks(input_data, target_data, input_size, output_size, layer_sizes_list, epochs)

# Display results
import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)

Benchmarking network with layer sizes: [128]


/var/folders/2m/zt75k_js249d_qcmywz2vb0r0000gp/T/ipykernel_45963/2836665056.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.make_functional` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.func.functional_call` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  fmodel, params = make_functional(model)
/var/folders/2m/zt75k_js249d_qcmywz2vb0r0000gp/T/ipykernel_45963/2836665056.py:50: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.jacfwd` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.func.jacfwd` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  gr

: 